Dépendances:
- opencv-python
- numpy
- scikit-learn
- imbalanced-learn

Pour lancer l'installation: 
pip install opencv-python numpy scikit-learn imbalanced-learn

In [21]:
import os
import cv2
import numpy as np
import webbrowser
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

In [22]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)  # Lire en niveaux de gris
        if img is not None:
            # Redimensionner l'image pour avoir une taille uniforme
            img = cv2.resize(img, (64, 64))
            images.append(img)
    return images

In [23]:
# Charger les images de chaque classe pour l'entraînement et les tests
train_normal_images = load_images_from_folder('./data/chest_Xray/train/NORMAL')
train_malade_images = load_images_from_folder('./data/chest_Xray/train/PNEUMONIA')
test_normal_images = load_images_from_folder('./data/chest_Xray/test/NORMAL')
test_malade_images = load_images_from_folder('./data/chest_Xray/test/PNEUMONIA')

# Créer les labels correspondants
train_normal_labels = ['normal'] * len(train_normal_images)
train_malade_labels = ['malade'] * len(train_malade_images)
test_normal_labels = ['normal'] * len(test_normal_images)
test_malade_labels = ['malade'] * len(test_malade_images)

# Combiner les images et les labels
images = train_normal_images + train_malade_images + test_normal_images + test_malade_images
labels = train_normal_labels + train_malade_labels + test_normal_labels + test_malade_labels

# Convertir les listes en tableaux numpy
X = np.array(images)
y = np.array(labels)

# Aplatir les images pour les rendre compatibles avec l'algorithme SVM
X = X.reshape((X.shape[0], -1))

# Standardiser les données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Séparer les jeux de données en entraînement et test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, stratify=y, random_state=42)

# Appliquer SMOTE pour équilibrer les classes dans les données d'entraînement
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Définir les hyperparamètres à tester pour GridSearchCV
param_grid = {
    'C': [30],
    'kernel': ['rbf'],
    'gamma': ['scale']
}

# Initialiser GridSearchCV avec le modèle SVM et les paramètres en intégrant la cross-validation
grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy')

# Entraîner GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Afficher les meilleurs hyperparamètres trouvés pour ajuster
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Utiliser le meilleur modèle trouvé pour faire des prédictions
best_model = grid_search.best_estimator_

# Afficher les résultats sur les données de test
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", report)

# Générer un fichier HTML pour afficher les résultats
html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Résultats de la Classification</title>
</head>
<body>
    <h1>Rapport de Classification SVM pour la Pneumonie</h1>
    <h2>Meilleurs Hyperparamètres</h2>
    <p>{best_params}</p>
    <h2>Précision</h2>
    <p>{accuracy:.4f}</p>
    <h2>Rapport de Classification</h2>
    <pre>{report}</pre>
</body>
</html>
"""

# Écrire le contenu HTML dans un fichier

html_dir = 'html_files'
html_file_path = os.path.join(html_dir, 'classification_report.html')

with open(html_file_path, 'w') as file:
    file.write(html_content)

print("Fichier HTML généré avec succès.")

# Ouvrir le fichier HTML dans le navigateur
webbrowser.open(f'file://{os.path.realpath(html_file_path)}')

Best parameters found:  {'C': 30, 'gamma': 'scale', 'kernel': 'rbf'}
Accuracy: 0.95
Classification Report:
               precision    recall  f1-score   support

      malade       0.96      0.97      0.97      1066
      normal       0.92      0.90      0.91       394

    accuracy                           0.95      1460
   macro avg       0.94      0.93      0.94      1460
weighted avg       0.95      0.95      0.95      1460

Le fichier HTML a été généré avec succès.


True

C: 30  
Gamma: scale
Kernel: rbf

Malade

    precision : Proportion de vrais positifs parmi les exemples classés comme "malade".
    recall : Proportion de vrais positifs parmi tous les exemples réellement "malade".
    f1-score : Moyenne harmonique de la précision et du rappel, fournissant un seul score pour évaluer le modèle.
    support : Nombre de vrais exemples de la classe "malade" dans l'ensemble de test.

Normal

    precision : Proportion de vrais positifs parmi les exemples classés comme "normal".
    recall : Proportion de vrais positifs parmi tous les exemples réellement "normal".
    f1-score : Moyenne harmonique de la précision et du rappel pour la classe "normal".
    support : Nombre de vrais exemples de la classe "normal" dans l'ensemble de test.